# Optimization notebook
This notebook is to run a multi-objective optimization with the Zambezi model. Two alternatives will be given to that end. First implementation is with the Platypus library which has a fairly simple interface. The second implementation is the EMA Workbench which additionally offers parallelization functionality

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../src')
from model_zambezi_OPT import model_zambezi

from platypus import NSGAII, Problem, Real

In [2]:
ZambeziProblem = model_zambezi()

## Platypus (NSGAII)

In [ ]:
problem = Problem(ZambeziProblem.Nvar, ZambeziProblem.Nobj)
problem.types[:] = Real(0, 1)
problem.function = ZambeziProblem.evaluate

algorithm = NSGAII(problem=problem, population_size=20)
algorithm.run(100)

In [3]:
objectives_outcome = dict()
for i, column_name in enumerate(['Hydropower','Environment','Irrigation']):
    objectives_outcome[column_name] = [s.objectives[i] for s in algorithm.result]

objectives_df = pd.DataFrame(objectives_outcome)


NameError: name 'algorithm' is not defined

In [4]:
print(objectives_df)
from src.various_plots import parallel_plots
parallel_plots(objectives_df)

NameError: name 'objectives_df' is not defined

## EMA Workbench

In [5]:
def model_wrapper(**kwargs):
    input = [kwargs['v' + str(i)] for i in range(len(kwargs))]
    Hydropower, Environment, Irrigation = tuple(ZambeziProblem.evaluate(np.array(input)))
    return Hydropower, Environment, Irrigation


In [6]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)

model = Model('zambeziproblem', function=model_wrapper)

model.levers = [RealParameter('v' + str(i), -1, 1) for i in range(ZambeziProblem.Nvar)]

#specify outcomes
model.outcomes = [ScalarOutcome('Hydropower', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Environment', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Irrigation', ScalarOutcome.MINIMIZE)]

## First trying with SequentialEvaluator

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])

## MultiprocessingEvaluator

In [16]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])

## Just giving different epsilon values with SequentialEvaluator

In [8]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1,]*len(model.outcomes))

  0%|                                                  | 0/100 [00:00<?, ?it/s]c:\Users\dakoluk\Documents\16.02.22.Multiobjective\Multiobjective-multi-reservoir-control-d50e4da0f6a9a9c852b4904e640299adc96714bb\ZambeziSmashPython\src\alternative_policy_structures.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  y = min(input_inflow,input_w*(pow(input_inflow/hdg_dn,m)))
c:\Users\dakoluk\Documents\16.02.22.Multiobjective\Multiobjective-multi-reservoir-control-d50e4da0f6a9a9c852b4904e640299adc96714bb\ZambeziSmashPython\src\alternative_policy_structures.py:33: RuntimeWarning: invalid value encountered in double_scalars
  y = min(input_inflow,input_w*(pow(input_inflow/hdg_dn,m)))
100%|████████████████████████████████████████| 100/100 [30:24<00:00, 18.24s/it]
[MainProcess/INFO] optimization completed, found 3 solutions


## Just giving different epsilon values with MultiprocessingEvaluator

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1,]*len(model.outcomes))

## IpyparallelEvaluator is not gonna work

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, IpyparallelEvaluator, ema_logging
from ema_workbench.em_framework.ema_ipyparallel import (start_logwatcher, set_engine_logger,
                                  initialize_engines, cleanup, _run_experiment)
import dill
from ipyparallel import Client


rc = Client()

ema_logging.log_to_stderr(ema_logging.INFO)

with IpyparallelEvaluator(model, client=rc) as evaluator:
    results = evaluator.optimize(nfe=100, searchover='levers',
    epsilons=[0.1,]*len(model.outcomes))